<a href="https://colab.research.google.com/github/BardRimon/Study/blob/main/algorithms_and_data_structures/sem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 3
Сравнительный анализ эффективности методов сортировки данных во внешней памяти.

Требования к функциональным возможностям программы.
Программа должна содержать меню, позволяющее выбирать один из двух режимов работы программы:
1)	сортировка файла данных, сформированных случайным образом;
2)	режим накопления статистических данных
В первом режиме требуется предоставить пользователю следующие возможности:
-	задавать размер числовой последовательности, содержащейся в файле;
-	указывать диапазон значений элементов последовательности;
-	выбирать метод внешней сортировки.
Результаты работы программы в данном режиме:
-	вывести на экран количество сравнений и перестановок элементов массива.
Во втором режиме пользователь должен иметь возможность:
-	выбирать способ формирования элементов последовательности, содержащейся в фай-ле (случайные значения, упорядоченная последовательность значений, значения рас-положены в обратном порядке);
-	задавать диапазон и шаг изменения размера последовательности;
-	выбирать метод внешней сортировки.
Результаты работы программы в данном режиме:
Для каждого значения размера формируется набор данных и сортируется выбранным мето-дом. В файл с указанным именем выводятся значения  времени сортировки для каждого ко-личества элементов в наборе.


Напомним, что
**Для получения оценки «отлично» необходимо программно реализовать следующие методы сортировки данных во внешней памяти: сортировка прямым слиянием; многофазная сортировка. Оценить быстродействие указанных методов и степень естественности их пове-дения.**

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import csv
import shutil

In [14]:
import csv
import os
import shutil
import tempfile

def _get_row_count(filename, delimiter=','):
    """Подсчитывает количество строк в CSV файле."""
    count = 0
    try:
        with open(filename, 'r', newline='', encoding='utf-8') as f:
            reader = csv.reader(f, delimiter=delimiter)
            for _ in reader:
                count += 1
    except FileNotFoundError:
        return 0
    return count

def _compare_keys(key1_str, key2_str):
    """
    Сравнивает два ключа. Предполагается, что ключи - это строковые представления чисел.
    Возвращает -1 если key1 < key2, 0 если key1 == key2, 1 если key1 > key2.
    """
    try:
        # В задаче указано "записи файла целые числа (ключи)"
        # Используем float для большей гибкости (например, если числа могут быть дробными)
        # Если гарантированно целые, можно использовать int()
        k1 = float(key1_str)
        k2 = float(key2_str)
        if k1 < k2:
            return -1
        if k1 > k2:
            return 1
        return 0
    except ValueError:
        # Аварийный вариант: если не удалось преобразовать в число, сравниваем как строки
        # Это не должно происходить при корректных входных данных согласно условию
        if key1_str < key2_str:
            return -1
        if key1_str > key2_str:
            return 1
        return 0

def _read_next_row_from_reader(reader):
    """Вспомогательная функция для чтения следующей строки, возвращает None при StopIteration."""
    try:
        return next(reader)
    except StopIteration:
        return None

def _split_file(source_filename, temp_file_b_name, temp_file_c_name, delimiter=',', key_column_index=0):
    """
    Фаза 1: Разделение.
    Читает из source_filename и попеременно распределяет записи в temp_file_b и temp_file_c.
    Параметр key_column_index здесь не используется, но добавлен для консистентности интерфейса.
    """
    # print(f"Разделение: {source_filename} -> ({temp_file_b_name}, {temp_file_c_name})")
    try:
        with open(source_filename, 'r', newline='', encoding='utf-8') as f_source, \
             open(temp_file_b_name, 'w', newline='', encoding='utf-8') as f_b, \
             open(temp_file_c_name, 'w', newline='', encoding='utf-8') as f_c:

            reader_source = csv.reader(f_source, delimiter=delimiter)
            writer_b = csv.writer(f_b, delimiter=delimiter)
            writer_c = csv.writer(f_c, delimiter=delimiter)

            use_b = True
            for row in reader_source:
                if use_b:
                    writer_b.writerow(row)
                else:
                    writer_c.writerow(row)
                use_b = not use_b
    except Exception as e:
        print(f"Ошибка во время разделения файла {source_filename}: {e}")
        raise

def _merge_files(target_filename, temp_file_b_name, temp_file_c_name, series_length,
                 total_records_in_source, delimiter=',', key_column_index=0):
    """
    Фаза 2: Слияние.
    Сливает отсортированные серии (порции) из temp_file_b и temp_file_c в target_filename.
    """
    # print(f"Слияние: ({temp_file_b_name}, {temp_file_c_name}) -> {target_filename} (длина серии: {series_length})")

    # Убедимся, что временные файлы существуют, даже если они пустые (например, если исходный файл был мал)
    for temp_file in [temp_file_b_name, temp_file_c_name]:
        if not os.path.exists(temp_file) or os.path.getsize(temp_file) == 0:
            # Создаем пустой файл, если он не существует или пуст, чтобы csv.reader не вызвал ошибку
            open(temp_file, 'a', encoding='utf-8').close()

    try:
        with open(target_filename, 'w', newline='', encoding='utf-8') as f_target, \
             open(temp_file_b_name, 'r', newline='', encoding='utf-8') as f_b, \
             open(temp_file_c_name, 'r', newline='', encoding='utf-8') as f_c:

            writer_target = csv.writer(f_target, delimiter=delimiter)
            reader_b = csv.reader(f_b, delimiter=delimiter)
            reader_c = csv.reader(f_c, delimiter=delimiter)

            row_b = _read_next_row_from_reader(reader_b)
            row_c = _read_next_row_from_reader(reader_c)

            # Общее количество записей, обработанных в этой фазе слияния
            # Используется для корректной обработки ситуаций, когда один из файлов B или C
            # заканчивается раньше, чем все записи из исходного файла (до разделения) будут обработаны.
            records_merged_total_pass = 0

            # Цикл продолжается, пока не будут обработаны все записи, которые были в исходном файле
            # перед разделением на B и C для текущего этапа.
            while records_merged_total_pass < total_records_in_source:
                count_b_series = 0  # Записей считано из B для текущей серии
                count_c_series = 0  # Записей считано из C для текущей серии

                # Слияние одной серии из B и одной серии из C
                # Этот внутренний цикл соответствует пунктам 4-7 алгоритма для одной порции.
                while True:
                    # Определяем, можно ли брать запись из B для текущей серии
                    can_take_b = row_b is not None and count_b_series < series_length
                    # Определяем, можно ли брать запись из C для текущей серии
                    can_take_c = row_c is not None and count_c_series < series_length

                    # Если обе серии исчерпаны (или соответствующие файлы закончились)
                    if not can_take_b and not can_take_c:
                        break # Завершаем слияние текущей порции

                    # Выбираем, какую запись писать (меньшую)
                    write_b_chosen = False
                    if can_take_b and can_take_c:
                        # Проверка на случай пустых строк или строк с недостаточным количеством столбцов
                        key_b_valid = row_b and key_column_index < len(row_b)
                        key_c_valid = row_c and key_column_index < len(row_c)

                        if key_b_valid and key_c_valid:
                            if _compare_keys(row_b[key_column_index], row_c[key_column_index]) <= 0:
                                write_b_chosen = True
                        elif key_b_valid: # Ключ C невалиден, берем из B
                            write_b_chosen = True
                        # elif key_c_valid: # Ключ B невалиден, берем из C (write_b_chosen остается False)
                        # else: # Оба ключа невалидны, прерываем (не должно случиться при can_take_b/c = True)
                        #     break

                    elif can_take_b: # Только из B можно взять
                        write_b_chosen = True
                    # elif can_take_c: # Только из C можно взять (write_b_chosen остается False)
                    # else: # Невозможно взять ниоткуда (уже обработано `if not can_take_b and not can_take_c`)
                    #     break

                    if write_b_chosen:
                        writer_target.writerow(row_b)
                        row_b = _read_next_row_from_reader(reader_b)
                        count_b_series += 1
                    elif can_take_c: # Если не выбрали B, и из C можно взять
                        writer_target.writerow(row_c)
                        row_c = _read_next_row_from_reader(reader_c)
                        count_c_series += 1
                    else: # Не удалось выбрать запись (например, оба ключа невалидны или оба файла закончились)
                        break

                    records_merged_total_pass += 1

                # Если оба исходных файла B и C полностью прочитаны, и мы обработали все записи
                if row_b is None and row_c is None and records_merged_total_pass >= total_records_in_source:
                    break

            # Дополнительная проверка и дозапись оставшихся строк, если total_records_in_source
            # было немного неточно или файлы B/C содержали больше записей, чем ожидалось
            # (хотя при правильной логике это не должно быть основным механизмом).
            # Пункт 8 алгоритма ("Тогда оставшиеся записи из другого файла пересылаются в файл А")
            # более релевантен для ситуации, когда один из файлов В или С полностью исчерпан,
            # а другой еще содержит данные. Этот блок гарантирует дозапись.
            while row_b is not None:
                writer_target.writerow(row_b)
                row_b = _read_next_row_from_reader(reader_b)
            while row_c is not None:
                writer_target.writerow(row_c)
                row_c = _read_next_row_from_reader(reader_c)

    except Exception as e:
        print(f"Ошибка во время слияния в файл {target_filename} из ({temp_file_b_name}, {temp_file_c_name}): {e}")
        raise

def external_merge_sort(input_filename, output_filename, delimiter=',', key_column_index=0):
    """
    Сортирует CSV файл, используя внешнюю двухфазную сортировку прямым слиянием.
    :param input_filename: Путь к входному CSV файлу.
    :param output_filename: Путь для отсортированного выходного CSV файла.
    :param delimiter: Разделитель в CSV.
    :param key_column_index: Индекс столбца для сортировки (0-based).
                             Предполагается, что этот столбец содержит числовые данные.
    """
    if not os.path.exists(input_filename):
        print(f"Ошибка: Входной файл '{input_filename}' не найден.")
        return

    total_records = _get_row_count(input_filename, delimiter)
    # print(f"Всего записей в '{input_filename}': {total_records}")

    if total_records == 0:
        # Создаем пустой выходной файл
        open(output_filename, 'w', encoding='utf-8').close()
        print("Входной файл пуст. Выходной файл создан пустым.")
        return

    if total_records == 1:
        # Если одна запись, она уже отсортирована. Копируем в выходной файл.
        try:
            shutil.copy(input_filename, output_filename)
            print("Входной файл содержит одну запись. Скопировано в выходной файл.")
        except Exception as e:
            print(f"Ошибка при копировании файла с одной записью: {e}")
        return

    # Создаем временную директорию для хранения файлов A, B, C
    # Это упрощает управление временными файлами и их очистку.
    temp_dir = tempfile.mkdtemp(prefix="ext_sort_")

    try:
        # Начальный файл A - это копия входного файла во временной директории
        # Это позволяет не изменять исходный input_filename до завершения.
        current_a_path = os.path.join(temp_dir, "A_initial.csv")
        shutil.copyfile(input_filename, current_a_path)

        series_length = 1 # Начальная длина серии

        while True:
            # Условие завершения сортировки: длина порции (серии) достигает n (total_records)
            # Если series_length >= total_records, значит, после текущего слияния
            # весь файл будет одной отсортированной серией.
            is_final_merge = (series_length >= total_records)

            # Имена для временных файлов B и C внутри временной директории
            temp_b_path = os.path.join(temp_dir, "B.csv")
            temp_c_path = os.path.join(temp_dir, "C.csv")

            # Фаза 1: Разделение current_a_path на B.csv и C.csv
            _split_file(current_a_path, temp_b_path, temp_c_path, delimiter, key_column_index)

            # Если current_a_path не был исходной копией (A_initial.csv),
            # а был результатом предыдущего слияния, его можно удалить.
            if current_a_path != os.path.join(temp_dir, "A_initial.csv"):
                os.remove(current_a_path)

            # Определяем, куда будет производиться слияние
            if is_final_merge:
                # Последнее слияние происходит непосредственно в output_filename
                merge_target_path = output_filename
            else:
                # Промежуточное слияние происходит в новый временный файл A
                merge_target_path = os.path.join(temp_dir, "A_merged_next.csv")

            # Фаза 2: Слияние B.csv и C.csv в merge_target_path
            _merge_files(merge_target_path, temp_b_path, temp_c_path, series_length,
                         total_records, delimiter, key_column_index)

            # Удаляем временные файлы B и C, так как они уже слиты
            os.remove(temp_b_path)
            os.remove(temp_c_path)

            # Файл, в который только что слили, становится исходным файлом A для следующего этапа
            current_a_path = merge_target_path

            if is_final_merge:
                break # Сортировка завершена

            series_length *= 2 # Удваиваем длину серии для следующего этапа
            # print(f"Следующая длина серии: {series_length}")

        print(f"Внешняя сортировка завершена. Отсортированные данные в файле: '{output_filename}'")

    except Exception as e:
        print(f"Произошла ошибка во время внешней сортировки: {e}")
        # Можно добавить логирование или другие действия по обработке ошибок
        # Если output_filename был частично записан, он может быть поврежден.
        raise # Перевыброс исключения для индикации сбоя
    finally:
        # Очистка временной директории со всеми ее содержимым
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
        # print(f"Временная директория {temp_dir} очищена.")


# --- Пример использования ---
if __name__ == "__main__":
    # Создаем тестовый CSV файл
    test_input_filename = "sample_data.csv"
    test_output_filename = "sorted_sample_data.csv"

    # Данные: числа в первом столбце, другая информация во втором
    data_to_write = [
        [5, "apple"], [2, "banana"], [8, "cherry"], [1, "date"],
        [9, "elderberry"], [4, "fig"], [7, "grape"], [3, "honeydew"],
        [6, "kiwi"], [0, "lemon"], [12, "mango"], [10, "nectarine"],
        [11, "orange"], [15, "papaya"], [13, "quince"], [14, "raspberry"]
    ]
    # data_to_write = [[5],[2],[8],[1],[9]] # Более простой случай из описания

    with open(test_input_filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerows(data_to_write)
    print(f"Создан тестовый файл: {test_input_filename} с {len(data_to_write)} записями.")

    # Выполняем внешнюю сортировку
    # Сортируем по первому столбцу (индекс 0)
    external_merge_sort(test_input_filename, test_output_filename, key_column_index=0)

    # Проверяем результат
    print(f"\nСодержимое отсортированного файла ({test_output_filename}):")
    sorted_data_read = []
    try:
        with open(test_output_filename, 'r', newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            for row in reader:
                print(row)
                sorted_data_read.append(row)

        # Дополнительная проверка корректности сортировки
        expected_keys = sorted([float(item[0]) for item in data_to_write])
        actual_keys = [float(item[0]) for item in sorted_data_read]
        if expected_keys == actual_keys:
            print("\nПроверка сортировки: УСПЕШНО.")
        else:
            print("\nПроверка сортировки: НЕУДАЧА.")
            print(f"Ожидаемые ключи: {expected_keys}")
            print(f"Фактические ключи: {actual_keys}")

    except FileNotFoundError:
        print(f"Выходной файл {test_output_filename} не найден. Сортировка могла завершиться с ошибкой.")

    # Тест с пустым файлом
    empty_input_filename = "empty_data.csv"
    empty_output_filename = "sorted_empty_data.csv"
    open(empty_input_filename, 'w', encoding='utf-8').close()
    print(f"\nТестирование с пустым файлом: {empty_input_filename}")
    external_merge_sort(empty_input_filename, empty_output_filename)
    if os.path.exists(empty_output_filename) and os.path.getsize(empty_output_filename) == 0:
        print("Тест с пустым файлом: УСПЕШНО.")
    else:
        print("Тест с пустым файлом: НЕУДАЧА.")

    # Тест с файлом из одной строки
    single_input_filename = "single_data.csv"
    single_output_filename = "sorted_single_data.csv"
    with open(single_input_filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([42, "answer"])
    print(f"\nТестирование с файлом из одной строки: {single_input_filename}")
    external_merge_sort(single_input_filename, single_output_filename)
    print(f"Содержимое отсортированного файла из одной строки ({single_output_filename}):")
    try:
        with open(single_output_filename, 'r', newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            for row in reader:
                print(row)
    except FileNotFoundError:
        print(f"Выходной файл {single_output_filename} не найден.")

    # Очистка тестовых файлов (можно закомментировать для проверки)
    # for f in [test_input_filename, test_output_filename,
    #             empty_input_filename, empty_output_filename,
    #             single_input_filename, single_output_filename]:
    #     if os.path.exists(f):
    #         os.remove(f)
    # print("\nТестовые файлы удалены.")

Создан тестовый файл: sample_data.csv с 16 записями.
Внешняя сортировка завершена. Отсортированные данные в файле: 'sorted_sample_data.csv'

Содержимое отсортированного файла (sorted_sample_data.csv):
['1', 'date']
['2', 'banana']
['3', 'honeydew']
['4', 'fig']
['5', 'apple']
['7', 'grape']
['8', 'cherry']
['0', 'lemon']
['6', 'kiwi']
['9', 'elderberry']
['10', 'nectarine']
['11', 'orange']
['12', 'mango']
['13', 'quince']
['14', 'raspberry']
['15', 'papaya']

Проверка сортировки: НЕУДАЧА.
Ожидаемые ключи: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0]
Фактические ключи: [1.0, 2.0, 3.0, 4.0, 5.0, 7.0, 8.0, 0.0, 6.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0]

Тестирование с пустым файлом: empty_data.csv
Входной файл пуст. Выходной файл создан пустым.
Тест с пустым файлом: УСПЕШНО.

Тестирование с файлом из одной строки: single_data.csv
Входной файл содержит одну запись. Скопировано в выходной файл.
Содержимое отсортированного файла из одной строки 

In [ ]:
def write_csv()


def direct_merge(input_file_name, output_file_name):
    # фаза слияния
    shutil.copy(input_file_name, "A.csv")
    n = 0
    with open(input_file_name, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            if row: # Проверяем, что строка не пустая
                n += 1

    def spliting():
        with open('A.csv', 'r') as a:
            read = csv.reader(a)
            step = 1
            for row in read:
                if step % 2 == 1:
                    with open('B.csv', 'a', newline='') as b:
                        writer = b.writer(b)
                        writer.writeof(row)
                else:
                    with open('C.csv', 'a', newline='') as c:
                        writer = c.writer(c)
                        writer.writeof(row)
                step += 1

    def mergering(window)
            with open('A.csv', 'w', newline='') as a:
                writer = csv.writer(a)

                with open('B.csv', 'w', newline='') as b:
                    reader = csv.reader(b)
                    for row in reader:
                        writer.writerow(row)


    i = 0
    while i <= n:



In [ ]:
def create_file(data):
    with open('input.csv', 'w', newline='') as f:
      write = csv.writer(f)
      for element in data:
          write.writerow(element)


def check_file():
    with open('output.csv', 'r') as f:
        read = csv.reader(f)
        l_orig = list(read)
        l_cher = l_orig.copy()
        l_cher.sort()
        if l_cher == l_orig:
            return True
        else:
            return False


In [ ]:
d = [[random.randint(1, 1000) for _ in range(11)]]
create_file(d)


In [ ]:
t = []



for i in range(100, 100001, 100):
    data = [random.randint(1, 1000) for _ in range(i)]
    create_file(data)
    time_s = time.time()
    #тут сортировка
    # direct_merge_sort('input.txt', 'output.txt')

    time_f = time.time()
    t.appent(time_f - time_s)




